In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import mplleaflet

from geopy.distance import geodesic

import os

import math

from k_medoids import KMedoidsWeighted

%matplotlib inline

In [2]:
data_dir = '../data/'
mamografo_dataset_name = 'RF- Mamógrafos.csv'
use_cols = ['COD_MUN','2018/Dez'] # obtendo os dados mais recentes
municipios_dataset_name = 'dados_mun.csv'
tqdm.pandas()

In [3]:
def dist_to_near_mamografo(row):
    '''
    Retorna a menor distancia a um mamografo, dada a latitude e longitude de uma cidade
    '''
    distancias = cities_mamografo[cities_mamografo.UF == row.UF].apply(lambda x: geodesic((x['LATITUDE'], x['LONGITUDE']), (row.LATITUDE, row.LONGITUDE)).km , axis=1)
    index_min = distancias.idxmin()
    near_city = cities_mamografo.iloc[index_min]
    distancia = geodesic((near_city.LATITUDE, near_city.LONGITUDE), (row.LATITUDE, row.LONGITUDE)).km
    return near_city.COD_MUN, distancia
def population_around_mamografo(codigo_mun):
    '''
    Retorna a soma das populacoes das cidades ao redor de um mamografo
    '''
    cidades_proximas = municipios[municipios.COD_PROX == codigo_mun]
    return cidades_proximas.POPULACAO.astype(int).sum()

In [4]:
# Lendo os dataframes:

mamografo = pd.read_csv(os.path.join(data_dir, mamografo_dataset_name), encoding='latin', index_col=False, 
                     error_bad_lines=False, low_memory=False, usecols=use_cols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_dataset_name), encoding='latin', index_col=0, 
                     error_bad_lines=False, low_memory=False)

municipios['COD_MUN'] = municipios['COD_MUN'].apply(lambda x: (int) ((x - x%10)/10))
cities_mamografo = pd.merge(mamografo[mamografo['2018/Dez'] > 0], municipios, on='COD_MUN', how='left')
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[1], axis=1), COD_PROX = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[0], axis=1))
estados = municipios.UF.unique()
distancia_max = 70
populacao_max = 100000

In [5]:
UF_analisado = 'SP'
cidades_no_estado = municipios[municipios.UF == UF_analisado]
proximos, distantes = cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max]
mamografo_no_estado = cities_mamografo[cities_mamografo.UF == UF_analisado]
mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.progress_apply(lambda cod: population_around_mamografo(cod)))
pop_ok, pop_n_ok = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]
mamografo_no_estado

,COD_MUN,2018/Dez,UF,NOME,POPULACAO,LATITUDE,LONGITUDE,CAPITAL,CODIGO_UF,POP_ATENDIDA
502,350010,1,SP,Adamantina,35048,-21.6820,-51.0737,0,35,146886
503,350070,1,SP,Agudos,36524,-22.4694,-48.9863,0,35,39043
504,350160,3,SP,Americana,229322,-22.7374,-47.3331,0,35,229322
505,350170,1,SP,AmÃ©rico Brasiliense,38202,-21.7288,-48.1147,0,35,57685
506,350190,2,SP,Amparo,70280,-22.7088,-46.7720,0,35,112870
...,...,...,...,...,...,...,...,...,...,...
675,355620,1,SP,Valinhos,120258,-22.9698,-46.9974,0,35,192808
676,355640,1,SP,Vargem Grande do Sul,41807,-21.8322,-46.8913,0,35,41807
677,355650,2,SP,VÃ¡rzea Paulista,116601,-23.2136,-46.8234,0,35,224921
678,355700,2,SP,Votorantim,117794,-23.5446,-47.4388,0,35,244394


In [6]:
plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com mamografos distantes
plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com mamografos proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='yellowgreen', s = 400) # plotar cidades em que o numero de mamografos sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='darkolivegreen', s = 400) #plotar cidades em que o numero de mamografos nao sao adequados
mplleaflet.show()

In [8]:
total_cidades, total_populacao, total_mamografos, ideal_mamografos,suficiente,cidades_proximas, cidades_distantes, centros_suficiente, centros_insuficiente = [],[],[],[],[],[],[],[],[]
for estado in estados:
    UF_analisado = estado
    cidades_no_estado = municipios[municipios.UF == estado]
    mamografo_no_estado = cities_mamografo[cities_mamografo.UF == estado]
    mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.apply(lambda cod: population_around_mamografo(cod)))
    
    distantes, proximas = cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max]
    suficientes, insuficientes = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]
    total_populacao_estado = cidades_no_estado.POPULACAO.astype(int).sum()
    total_mamografo_estado = mamografo_no_estado['2018/Dez'].astype(int).sum()
    ideal_mamografo_estado = math.ceil(total_populacao_estado/populacao_max)
    
    total_cidades.append(cidades_no_estado.COD_MUN.count())
    total_populacao.append(total_populacao_estado)
    total_mamografos.append(total_mamografo_estado)
    ideal_mamografos.append(ideal_mamografo_estado)
    suficiente.append(total_mamografo_estado > ideal_mamografo_estado)
    cidades_proximas.append(proximas.COD_MUN.count())
    cidades_distantes.append(distantes.COD_MUN.count())
    centros_suficiente.append(suficientes.COD_MUN.count())
    centros_insuficiente.append(insuficientes.COD_MUN.count())
    
panorama_distribuicao_mamografos_uf = pd.DataFrame({
    'UF': estados,
    'TOTAL_CIDADES': total_cidades,
    'TOTAL_POPULACAO': total_populacao,
    'TOTAL_MAMOGRAGOS': total_mamografos,
    'IDEAL_MAMOGRAFOS': ideal_mamografos,
    'SUFICIENTE': suficiente,
    'CIDADES_PROXIMAS': cidades_proximas,
    'CIDADES_DISTANTES': cidades_distantes,
    'CENTROS_SUFICIENTES': centros_suficiente,
    'CENTROS_INSUFICIENTES': centros_insuficiente
},columns= ['UF', 'TOTAL_MAMOGRAGOS', 'TOTAL_POPULACAO', 'IDEAL_MAMOGRAFOS','SUFICIENTE','TOTAL_CIDADES', 'CIDADES_DISTANTES' , 'CIDADES_PROXIMAS', 'CENTROS_SUFICIENTES', 'CENTROS_INSUFICIENTES'])
panorama_distribuicao_mamografos_uf

,UF,TOTAL_MAMOGRAGOS,TOTAL_POPULACAO,IDEAL_MAMOGRAFOS,SUFICIENTE,TOTAL_CIDADES,CIDADES_DISTANTES,CIDADES_PROXIMAS,CENTROS_SUFICIENTES,CENTROS_INSUFICIENTES
0,RO,14,1768204,18,False,52,29,23,0,5
1,AC,3,803513,9,False,22,14,8,0,2
2,AM,55,3938336,40,True,62,15,47,29,7
3,RR,4,505665,6,False,15,12,3,0,1
4,PA,50,8206923,83,False,144,44,100,1,22
5,AP,2,766679,8,False,16,13,3,0,1
6,TO,18,1515126,16,True,139,80,59,1,6
7,MA,30,6904241,70,False,217,92,125,0,15
8,PI,40,3204028,33,True,224,81,143,3,10
9,CE,69,8904459,90,False,184,14,170,1,24


In [7]:
# panorama_distribuicao_mamografos_uf.CIDADES_DISTANTES / panorama_distribuicao_mamografos_uf.TOTAL_CIDADES
# aplicar o k-medoids para definir novos pontos
UF_analisado = 'PA'
cidades_no_estado = municipios[municipios.UF == UF_analisado]
cidades_com_mamografo = cities_mamografo[cities_mamografo.UF == UF_analisado]['COD_MUN'].values.tolist()
X = cidades_no_estado[['LATITUDE', 'LONGITUDE', 'COD_MUN']].values.tolist()
medoids_fix_idxs = []
for i in range(len(X)):
    cod_mun = X[i][2]
    if cod_mun in cidades_com_mamografo:
        medoids_fix_idxs.append(i)

# quantidade inicial de cidades com mamografos
n_centros = len(medoids_fix_idxs)

# quantidade de novas cidades com novo centro
n_novos = 1
clustering = KMedoidsWeighted(n_clusters=n_centros+n_novos)
medoids_idxs, clusters = clustering.fit(X, medoids_fix_idxs, max_itr=10, verbose=1)

Iteration: 0 ... loss: 78.91866096010644
Iteration: 1 ... loss: 75.36568850938481


In [8]:
# adicionando cidades sugeridas na lista de cidades com mamografos
novos_medoids = np.setdiff1d(medoids_idxs, medoids_fix_idxs)
for i in novos_medoids:
    cod_mun = int(X[i][2])
    nova_cidade = municipios[municipios.COD_MUN == cod_mun].values.tolist()[0]
    cities_mamografo = cities_mamografo.append({'COD_MUN' : cod_mun,
                                                '2018/Dez': 0,
                                                'UF': nova_cidade[0],
                                                'NOME': nova_cidade[2],
                                                'POPULACAO': nova_cidade[3],
                                                'LATITUDE': nova_cidade[4],
                                                'LONGITUDE': nova_cidade[5],
                                                'CAPITAL': nova_cidade[6],
                                                'CODIGO_UF': nova_cidade[7]
                                               } , ignore_index=True)

In [18]:
# atualizando distancias no municipio modificado
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: (dist_to_near_mamografo(row)[1] if (row.UF == UF_analisado) else row.DISTANCIAS), axis=1), COD_PROX = municipios.progress_apply(lambda row: (dist_to_near_mamografo(row)[0] if (row.UF == UF_analisado) else row.COD_PROX), axis=1))
mamografo_no_estado = cities_mamografo[cities_mamografo.UF == UF_analisado]
mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.apply(lambda cod: population_around_mamografo(cod)))
mamografo_no_estado = mamografo_no_estado.assign(IDEAL_MAMOGRAFO = mamografo_no_estado.POP_ATENDIDA.apply(lambda pop: math.ceil(pop/populacao_max)))


In [19]:
mamografo_no_estado

,COD_MUN,2018/Dez,UF,NOME,POPULACAO,LATITUDE,LONGITUDE,CAPITAL,CODIGO_UF,POP_ATENDIDA,IDEAL_MAMOGRAFO
44,150060,1,PA,Altamira,108382,-3.20407,-52.2100,0,15,276744,3
45,150080,2,PA,Ananindeua,505404,-1.36391,-48.3743,0,15,552060,6
46,150130,2,PA,Barcarena,115779,-1.51187,-48.6195,0,15,502941,6
47,150140,14,PA,BelÃ©m,1439561,-1.45540,-48.4898,1,15,1493625,15
48,150150,1,PA,Benevides,58637,-1.36183,-48.2434,0,15,186083,2
49,150170,2,PA,BraganÃ§a,121528,-1.06126,-46.7826,0,15,266791,3
50,150180,1,PA,Breves,98231,-1.68036,-50.4791,0,15,330329,4
51,150210,1,PA,CametÃ¡,130868,-2.24295,-49.4979,0,15,319680,4
52,150220,4,PA,Capanema,66353,-1.20529,-47.1778,0,15,219070,3
53,150230,1,PA,CapitÃ£o PoÃ§o,52693,-1.74785,-47.0629,0,15,318061,4
